### Implement the next strategies in steps: ###

- **Fill in missing weather data**
    Original code from Kaggle notebook :
    https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling by @aitude


- **Data cleanup**
     Use data cleanup strategy from Kaggle notebook : 
     https://www.kaggle.com/purist1024/ashrae-simple-data-cleanup-lb-1-08-no-leaks by @purist1024
     
     - Drop rows with unjustified zeros
     - Site0 rows - the first 141 rows
     - Abnormally high reading from building 1099
     - Zero readings from electrical meters   


- **Feature engineering**
    - Time features: weekday, weekend, month, is_holiday
    - Feels Like temperature
    

- **LightGBM**
    - Use Bayesian optimization for parameters tuning

In [74]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import datetime
from meteocalc import feels_like
import gc
from bayes_opt import BayesianOptimization
from sklearn.model_selection import TimeSeriesSplit
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = "../../Ashrae/data/"

In [86]:
train_df = pd.read_csv(DATA_PATH + 'train.csv')
building_df = pd.read_csv(DATA_PATH + 'building_metadata.csv')
weather_df = pd.read_csv(DATA_PATH + 'weather_train.csv')

test_df = pd.read_csv(DATA_PATH + 'test.csv')
weather_test_df = pd.read_csv(DATA_PATH + 'weather_test.csv')

**1. Fill in missing weather data**
Original code from:  https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling by @aitude

In [87]:
# Original code from https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling by @aitude

def fill_weather_dataset(weather_df):
    
    # Find Missing Dates
    time_format = "%Y-%m-%d %H:%M:%S"
    start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
    end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

    missing_hours = []
    for site_id in range(16):
        site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_df = pd.concat([weather_df,new_rows])

        weather_df = weather_df.reset_index(drop=True)           

    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["day"] = weather_df["datetime"].dt.weekday
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    weather_df["hour"] = weather_df["datetime"].dt.hour
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])

    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)

    # Step 1
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

    weather_df.update(cloud_coverage_filler,overwrite=False)

    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)

    # Step 1
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime', 'week'],axis=1)
        
    return weather_df

In [88]:
def get_meteorological_features(data):
        def calculate_rh(df):
            df['relative_humidity'] = 100 * (np.exp((17.625 * df['dew_temperature']) / (243.04 + df['dew_temperature'])) / np.exp((17.625 * df['air_temperature'])/(243.04 + df['air_temperature'])))
        def calculate_fl(df):
            flike_final = []
            flike = []
            # calculate Feels Like temperature
            for i in range(len(df)):
                at = df['air_temperature'][i]
                rh = df['relative_humidity'][i]
                ws = df['wind_speed'][i]
                flike.append(feels_like(at, rh, ws))
            for i in range(len(flike)):
                flike_final.append(flike[i].f)
            df['feels_like'] = flike_final
            del flike_final, flike, at, rh, ws
        calculate_rh(data)
        calculate_fl(data)
        return data

In [89]:
weather_df = fill_weather_dataset(weather_df)
weather_test_df = fill_weather_dataset(weather_test_df)

In [90]:
weather_df = get_meteorological_features(weather_df)
weather_test_df = get_meteorological_features(weather_test_df)

**2. Reduce memory usage**
Original code from: https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin

In [91]:
# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [92]:
train_df = reduce_mem_usage(train_df,use_float16=True)
building_df = reduce_mem_usage(building_df,use_float16=True)
weather_df = reduce_mem_usage(weather_df,use_float16=True)

test_df = reduce_mem_usage(test_df, use_float16 = True)
weather_test_df = reduce_mem_usage(weather_test_df, use_float16 = True)

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.90 MB
Decreased by 71.8%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.8%
Memory usage of dataframe is 15.01 MB
Memory usage after optimization is: 3.60 MB
Decreased by 76.0%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.65 MB
Decreased by 71.8%
Memory usage of dataframe is 29.94 MB
Memory usage after optimization is: 7.17 MB
Decreased by 76.0%


**3. Merge data**

In [93]:
train_df = train_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])

test_df = test_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
test_df = test_df.merge(weather_test_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])

del weather_df, weather_test_df
gc.collect()

20

**4. Data Cleanup**
Use rows_to_drop dataset from:  https://www.kaggle.com/purist1024/ashrae-simple-data-cleanup-lb-1-08-no-leaks by @purist1024

In [94]:
# Read in rows_to_drop.csv

rows_to_drop = pd.read_csv(DATA_PATH + 'rows_to_drop.csv')

In [95]:
train_df.drop(rows_to_drop.loc[:, '0'], inplace = True)
train_df.reset_index(drop = True, inplace = True)

**5. Feature engineering**

In [98]:
def features_engineering(df):
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["weekday"] = df["timestamp"].dt.weekday
    df["weekend"] = df['weekday'].apply(lambda day_: 1 if day_>5 else 0)
    
    holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                    "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                    "2017-01-02", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                    "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                    "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                    "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                    "2019-01-01"]
    
    df["is_holiday"] = (df.timestamp.isin(holidays)).astype(int)
    df['square_feet'] =  np.log1p(df['square_feet'])
    
    # Remove Unused Columns
    drop = ["timestamp"]
    df = df.drop(drop, axis=1)
    gc.collect()
    
    # Encode Categorical Data
    le = LabelEncoder()
    df["primary_use"] = le.fit_transform(df["primary_use"])
    
    return df

In [99]:
train_df = features_engineering(train_df)
test_df = features_engineering(test_df)

In [100]:
train_df.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,day,month,...,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,relative_humidity,feels_like,weekday,weekend,is_holiday
0,0,0,0.000000,0,0,8.913685,2008.0,NaN,4,1,...,2.689453,1019.5,0.0,0.000000,0,73.8125,25.000000,4,0,0
1,105,0,23.303600,1,0,10.832181,NaN,5.0,4,1,...,0.041656,1021.0,240.0,3.099609,0,90.5625,-2.794922,4,0,0
2,106,0,0.374600,1,0,8.589514,NaN,4.0,4,1,...,0.041656,1021.0,240.0,3.099609,0,90.5625,-2.794922,4,0,0
3,106,3,0.000000,1,0,8.589514,NaN,4.0,4,1,...,0.041656,1021.0,240.0,3.099609,0,90.5625,-2.794922,4,0,0
4,107,0,175.184006,1,0,11.487946,2005.0,10.0,4,1,...,0.041656,1021.0,240.0,3.099609,0,90.5625,-2.794922,4,0,0


In [101]:
train_df.to_csv('train_final.csv')
test_df.to_csv('test_final.csv')

In [104]:
train_df.isna().sum()

building_id                  0
meter                        0
meter_reading                0
site_id                      0
primary_use                  0
square_feet                  0
year_built            11685636
floor_count           15811110
day                          0
month                        0
air_temperature              0
cloud_coverage               0
dew_temperature              0
precip_depth_1_hr            0
sea_level_pressure           0
wind_direction               0
wind_speed                   0
hour                         0
relative_humidity            0
feels_like                   0
weekday                      0
weekend                      0
is_holiday                   0
dtype: int64

In [134]:
test_df.isna().sum()

row_id                       0
building_id                  0
meter                        0
site_id                      0
primary_use                  0
square_feet                  0
year_built            24598080
floor_count           34444320
day                          0
month                        0
air_temperature              0
cloud_coverage               0
dew_temperature              0
precip_depth_1_hr            0
sea_level_pressure           0
wind_direction               0
wind_speed                   0
hour                         0
relative_humidity            0
feels_like                   0
weekday                      0
weekend                      0
is_holiday                   0
dtype: int64

In [135]:
# The two column with null values do not have a high feature importance, so try dropping them

col_to_drop = ['year_built', 'floor_count']

train = train_df.drop(columns = col_to_drop)
test = test_df.drop(columns = col_to_drop)